In [ ]:
# import depencies
from scipy.sparse import csr_matrix
import tables
import h5py
import hdf5plugin

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import seaborn as sb
from scipy import signal
import glob

import pandas as pd

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl

# scale figure
mpl.rcParams['figure.dpi'] = 200

# Import contact frequency matrices


In [ ]:
# import raw data files

file_names = []
for name in glob.glob('./Raw data/*_5kb.h5'):
    file_names.append(name)

In [ ]:
file_names

# Define functions useful for analysis

In [ ]:
# import HiC heat map data as a matrix

#argument:
# filename: input name of the file you wish to import matrix for

#output:
#the heat map itself as a matrix
def make_matrix(filename):
    with tables.open_file(filename, 'r') as f:
        parts = {}
        try:
            for matrix_part in ('data', 'indices', 'indptr', 'shape'):
                parts[matrix_part] = getattr(f.root.matrix, matrix_part).read()
        except Exception as e:
            log.info('No h5 file. Please check parameters concerning the file type!')
            e
        matrix = csr_matrix(tuple([parts['data'], parts['indices'], parts['indptr']]),
                            shape=parts['shape'])
        matrix_array = matrix.toarray()
    return matrix_array

In [ ]:
#import interval information, contains list of indices and chromosome that they correspond to

#argument:
# filename: input name of the file you wish to import intervals for

#output:
#interval_list: list of chromosomes, indices
#keychr: keys to access different list from interval_list

def get_intervals(filename):
    file = h5py.File(filename)
    key_list = file['intervals'].keys()
    interval_list = {}
    for key in key_list:
        interval_list[key] = file['intervals'][key][()]
    keychr = list(key_list)
    return interval_list, keychr

In [ ]:
#generate a matrix for specific chromosome vs chromosome interaction

#argument:
#ma: matrix of heatmap
#chromosome_x: chromosome to use for x axis
#chromosome_y: chromosome to use for y axis

#output:
#matrix of desired region

def call_region(ma,chromosome_x,chromosome_y):
    start_x = indices[chromosome_x]['start']
    end_x = indices[chromosome_x]['end']
    
    start_y = indices[chromosome_y]['start']
    end_y = indices[chromosome_y]['end']
    
    return  ma[start_x:end_x,start_y:end_y]

In [ ]:
# chunky code to get the end index for regions, requried for get_indices function
# argument:
# chrlist: full list of which chromosome each index correspond to

#output:
#end: end index for each chromosome

def get_end_idx(chrlist):
    start = {}
    end = {}
    i = 0
    for chromes in chrlist:
        if chromes == b'chrIV':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXV':
            end[chromes] = i
            i+=1
        elif chromes == b'chrVII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXVI':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXIII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXIV':
            end[chromes] = i
            i+=1
        elif chromes == b'chrX':
            end[chromes] = i
            i+=1
        elif chromes == b'chrXI':
            end[chromes] = i
            i+=1
        elif chromes == b'chrV':
            end[chromes] = i
            i+=1
        elif chromes == b'chrVIII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrIX':
            end[chromes] = i
            i+=1
        elif chromes == b'chrIII':
            end[chromes] = i
            i+=1
        elif chromes == b'chrVI':
            end[chromes] = i
            i+=1
        elif chromes == b'chrI':
            end[chromes] = i
            i+=1
        elif chromes == b'chrM':
            end[chromes] = i
            i+=1
        else:
            break
    return end

In [ ]:
# makes a dictionary of each chromosomes start and end index

# argument:
# chromosomes: the list of chromosomes
# end_idx: output of get_end_idx

#output:
#idxs: dictionary of each chromosomes start and end index

def get_indices(chromosomes,end_idx):
    idxs = {}
    next_start = 0
    for chromes in chromosomes:
        idxs[chromes] = {}
        idxs[chromes]['end'] = end_idx[chromes]+1

        if chromes == str(b'chrIV'):
            idxs[chromes]['start'] = 0
        else:
            idxs[chromes]['start'] = next_start
        next_start = end_idx[chromes]+1
        
    return idxs

In [ ]:
#divide 2 matrices

#argument: 
#nom_matrix: matrix on the nominator
#denom_matrix: matrix on the denominator
#log2: whether to apply log2 to output matrix

#output:
#matrix: matrix after division

def divide_matrices(nom_matrix, denom_matrix, log2 = True):
    divide_matrix = nom_matrix/denom_matrix
#     divide_matrix[~np.isfinite(divide_matrix)] = 0
    if log2:
        matrix = np.log2(divide_matrix)
        matrix[~np.isfinite(matrix)] = 0
    else:
        matrix = divide_matrix
        matrix[~np.isfinite(matrix)] = 0
    return matrix

In [ ]:
#make the triangular matrix a full matrix by duplicating its diagonal counterpart

#argument:
#ma: matrix of heatmap

#output:
# full_matrix

def make_symmetric(ma):
    rotmatrix = np.transpose(ma)
    np.fill_diagonal(rotmatrix,0)
    full_matrix = ma + rotmatrix
    return full_matrix

In [ ]:
# import h5 files and produce the full matrix

#argument:
#filename: file name of the data to import

#output:
# full_matrix

def make_symm_matrix(filename):
    with tables.open_file(filename, 'r') as f:
        parts = {}
        try:
            for matrix_part in ('data', 'indices', 'indptr', 'shape'):
                parts[matrix_part] = getattr(f.root.matrix, matrix_part).read()
        except Exception as e:
            log.info('No h5 file. Please check parameters concerning the file type!')
            e
        matrix = csr_matrix(tuple([parts['data'], parts['indices'], parts['indptr']]),
                            shape=parts['shape'])
        matrix_array = matrix.toarray()
        
        full_ma = make_symmetric(matrix_array)

    return full_ma

In [ ]:
# 2d filter the matrix, eliminate high frequency noise

#argument:
#ma: matrix
#filter_length: length to apply the filter, larger length decreases max signal frequency

#output:
#filtered_ma: matrix after apply filtering

def filtering(ma, filter_length = 5):
    filtered_ma = signal.spline_filter(ma,lmbda=filter_length)
    return filtered_ma

In [ ]:
# plot the matrix as a heatmap

def plot_matrix(ma, filt = None,filter_len=0, symmetric = True, hlim = None,vlim =None, center=False, interpolation =None, cmap = 'RdYlBu_r', fig_size = 15,vcenter=None, vmin = None, vmax = None ):
    fig, ax = plt.subplots()

    ma[~np.isfinite(ma)] = 0
    if filt:
        filtered_ma = filtering(ma,filter_length = filter_len)
    else:
        filtered_ma = ma
    if symmetric:
        full_ma = filtered_ma
    else:
        full_ma = make_symmetric(filtered_ma)
    if center:
        norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)
        im = ax.imshow(full_ma,cmap = cmap,norm=norm)
    else:
        im = ax.imshow(full_ma,cmap = cmap,interpolation = interpolation,vmin = vmin, vmax = vmax)
    plt.ylim(vlim)
    plt.xlim(hlim)
    cb = fig.colorbar(im,fraction=0.046, pad=0.04)
#     cb.set_ticks([0,1,2,3,4])
#     cb.set_ticklabels(['$10^0$','$10^1$','$10^2$','$10^3$','$10^4$'])
#     plt.xlabel('sites /5kB')
#     plt.ylabel('sites /5kB')

In [ ]:
# use this to average only the nonzero values. To include also empty values, use np.average(ma)
# note this can also be done for 1d array
#argument:
#ma: matrix of interest

#output:
# averaged value of matrix
def finite_average(ma):
    return np.sum(ma)/np.count_nonzero(ma)

# Import data

In [ ]:
#get the interval and key information, choose which file to use with file_names

interval ,keychr = get_intervals(file_names[0])
print(interval)
print(keychr)

In [ ]:
# list of chromosomes
chromosomes = [b'chrIV',b'chrXV',b'chrVII',b'chrXII',b'chrXVI',b'chrXIII',b'chrII',b'chrXIV',b'chrX',b'chrXI',b'chrV',b'chrVIII',b'chrIX',b'chrIII',b'chrVI',b'chrI',b'chrM']

In [ ]:
# get start and end indices corresponding to each chromosomes

indices = get_indices(chromosomes,get_end_idx(interval[keychr[0]]))

# print(indices)

In [ ]:
# import list of genes from rna-seq analysis
ds2 = pd.read_csv('ds2h_idx.txt', delimiter='\t')

# indices of of genes from rna-seq analysis
chr_idx = ds2.groupby('chromosome_name')['idx'].apply(list).to_dict()

#list of chromosomes
chr_list = [b'chrI', b'chrII', b'chrIII', b'chrIV', b'chrV', b'chrVI', b'chrVII', b'chrVIII', b'chrIX', b'chrX', b'chrXI', b'chrXII', b'chrXIII', b'chrXIV', b'chrXV', b'chrXVI',b'chrM']

chr_idx2 = dict(zip(chr_list, list(chr_idx.values())))

In [ ]:
# convenience definitions to label and index chromosomes
chrome_plots = ['IV','XV','VII','XII','XVI','XIII','II','XIV','X','XI','V','VIII','IX','III','VI','I']

chromosomes_num= [4, 15, 7, 12, 16, 13, 2, 14, 10, 11, 5, 8, 9, 3, 6, 1]

chromosomes_interval = []
for i in indices.keys():
    if i != b'chrM':
        inter = round((indices[i]['start'] + indices[i]['start'])/2)
        chromosomes_interval.append(inter)
end = indices[b'chrM']['end']

chromosome_label = []
for i in chromosomes_num:
    chromosome_label.append(str(i))
# chromosome_label.append('M')

In [ ]:
# get symmetric matrices from the files, naming nomenclature example: AA22 stands for Anchor Away 1st replicate of 2 hour time point

AA20 = make_symm_matrix(file_names[5])
AA22 = make_symm_matrix(file_names[3])
AA25 = make_symm_matrix(file_names[6])
AA30 = make_symm_matrix(file_names[4])
AA32 = make_symm_matrix(file_names[2])
AA35 = make_symm_matrix(file_names[7])
AA40 = make_symm_matrix(file_names[0])
AA42 = make_symm_matrix(file_names[1])
AA45 = make_symm_matrix(file_names[8])

# AA20 = make_symm_matrix(file_names[0])
# AA22 = make_symm_matrix(file_names[2])
# AA25 = make_symm_matrix(file_names[3])
# AA30 = make_symm_matrix(file_names[5])
# AA32 = make_symm_matrix(file_names[8])
# AA35 = make_symm_matrix(file_names[4])
# AA40 = make_symm_matrix(file_names[7])
# AA42 = make_symm_matrix(file_names[1])
# AA45 = make_symm_matrix(file_names[6])

# Ratio of different heatmaps

In [ ]:
# log2 ratio of different replicates and time points
AA22_AA20 = divide_matrices(AA22,AA20,True)
AA25_AA20 = divide_matrices(AA25,AA20,True)

AA32_AA30 = divide_matrices(AA32,AA30,True)
AA35_AA30 = divide_matrices(AA35,AA30,True)

AA42_AA40 =  divide_matrices(AA42,AA40,True)
AA45_AA40 = divide_matrices(AA45,AA40,True)

AA40_AA30 =  divide_matrices(AA40,AA30,True)
AA40_AA20 = divide_matrices(AA40,AA20,True)

# ratio of different replicates and time points
AA22_AA20_nl = divide_matrices(AA22,AA20,False)
AA25_AA20_nl = divide_matrices(AA25,AA20,False)

AA32_AA30_nl = divide_matrices(AA32,AA30,False)
AA35_AA30_nl = divide_matrices(AA35,AA30,False)

AA42_AA40_nl =  divide_matrices(AA42,AA40,False)
AA45_AA40_nl = divide_matrices(AA45,AA40,False)



## Averaging matrices

In [ ]:
# average heatmaps for 0 hour time point

AAave = np.zeros((2441, 2441))
for r in np.arange(0, 2441,1):
    for c in np.arange(0, 2441, 1):
        nom = np.array([ AA20[r, c], AA30[r, c], AA40[r,c]])
        denom = np.count_nonzero(nom) #count how many data points in nom are not zero
        if denom != 0:
            ave = np.sum(nom)/denom
            AAave[r, c] = ave
# turn anything not finite into 0 in the full matrix        
AAave[~np.isfinite(AAave)] = 0

In [ ]:
# average heatmaps for 2 hour time point


AA2ave = np.zeros((2441, 2441))
for r in np.arange(0, 2441,1):
    for c in np.arange(0, 2441, 1):
        nom = np.array([AA22[r, c], AA32[r, c], AA42[r,c]])
        denom = np.count_nonzero(nom) #count how many data points in nom are not zero
        if denom != 0:
            ave = np.sum(nom)/denom
            AA2ave[r, c] = ave
# turn anything not finite into 0 in the full matrix        
AA2ave[~np.isfinite(AA2ave)] = 0

In [ ]:
# average heatmaps for 5 hour time point


AA5ave = np.zeros((2441, 2441))
for r in np.arange(0, 2441,1):
    for c in np.arange(0, 2441, 1):
        nom = np.array([AA25[r, c], AA35[r,c], AA45[r,c]])
        denom = np.count_nonzero(nom) #count how many data points in nom are not zero
        if denom != 0:
            ave = np.sum(nom)/denom
            AA5ave[r, c] = ave
# turn anything not finite into 0 in the full matrix        
AA5ave[~np.isfinite(AA5ave)] = 0

In [ ]:
# ratio of 2 hour/DMSO and 5hour/DMSO

AAave2_0 = divide_matrices(AA2ave,AAave,log2=True)
AAave5_0 = divide_matrices(AA5ave,AAave,log2=True)

## Plotting log10 averaged

In [ ]:
#plot settings

mpl.rc('xtick', labelsize=12) 
mpl.rc('ytick', labelsize=12)
mpl.rc('figure',figsize = [10.0, 10.0])
# matplotlib.rc('axes',titlesize='large')

font = {'size'   : 12}

mpl.rc('font', **font)

from matplotlib.ticker import LogFormatter 
from matplotlib.ticker import LogFormatterExponent # <-- one new import here

In [ ]:
# DMSO
plot_matrix(np.log10(AAave)[:end,:end],cmap='Oranges',center = False,vmin =0, vmax = 4 )
plt.xlabel(None)
plt.ylabel(None)

# cb.ax.set_yticklabels(['a','b','c','d','e'])

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)
plt.title('DMSO Contact Map')

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

# plt.savefig('hic234plots/figures/AAave_log10.png')

In [ ]:
# 2hr

plot_matrix(np.log10(AA2ave)[:end,:end],cmap='Oranges',center = False,vmin =0, vmax = 4 )
plt.xlabel(None)
plt.ylabel(None)

# cb.ax.set_yticklabels(['a','b','c','d','e'])

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)

plt.title('2h Contact Map')

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

# plt.savefig('hic234plots/figures/AA2ave_log10.png')

In [ ]:
# 5hr

plot_matrix(np.log10(AA5ave)[:end,:end],cmap='Oranges',center = False,vmin =0, vmax = 4 )
plt.xlabel(None)
plt.ylabel(None)

# cb.ax.set_yticklabels(['a','b','c','d','e'])

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)

plt.title('5h Contact Map')

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

# plt.savefig('hic234plots/figures/AA5ave_log10.png')

## log2ratio replicate maps

In [ ]:
# Heatmap of ratio between different time points for different replicates

In [ ]:

plot_matrix(AA22_AA20,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)
plt.xlabel(None)
plt.ylabel(None)

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)

plt.title('2hr/DMSO Log2 Ratio AA2')

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

# plt.savefig('hic234plots/figures/AA22vAA20.png')

In [ ]:

plot_matrix(AA25_AA20,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)
plt.xlabel(None)
plt.ylabel(None)

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)

plt.title('5hr/DMSO Log2 Ratio AA2')

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

# plt.savefig('hic234plots/figures/AA25vAA20.png')

In [ ]:

plot_matrix(AA32_AA30,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)
plt.xlabel(None)
plt.ylabel(None)

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)

plt.title('2hr/DMSO Log2 Ratio AA3')

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

# plt.savefig('hic234plots/figures/AA32vAA30.png')

In [ ]:
plot_matrix(AA35_AA30,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)
plt.xlabel(None)
plt.ylabel(None)

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)

plt.title('5hr/DMSO Log2 Ratio AA3')

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

# plt.savefig('hic234plots/figures/AA35vAA30.png')

In [ ]:
plot_matrix(AA42_AA40,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)
plt.xlabel(None)
plt.ylabel(None)

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)

plt.title('2hr/DMSO Log2 Ratio AA4')

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

# plt.savefig('hic234plots/figures/AA42vAA40.png')

In [ ]:
plot_matrix(AA45_AA40,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)
plt.xlabel(None)
plt.ylabel(None)

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)

plt.title('5hr/DMSO Log2 Ratio AA4')

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

# plt.savefig('hic234plots/figures/AA45vAA40.png')

------

## Averaged heat map log2ratio

In [ ]:
# Heat map ratio between time points averaged between the replicates

In [ ]:
plot_matrix(AAave2_0,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)
plt.xlabel(None)
plt.ylabel(None)

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)

plt.title('2hr/DMSO Log2 Ratio Averaged')

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

# plt.savefig('hic234plots/figures/AAave2_0.png')

In [ ]:
plot_matrix(AAave5_0,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)
plt.xlabel(None)
plt.ylabel(None)

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)

plt.title('5hr/DMSO Log2 Ratio Averaged')

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

# plt.savefig('hic234plots/figures/AAave5_0.png')

## Heatmaps of specific chromosome regions

In [ ]:
plot_matrix(np.log10(AAave),cmap='Oranges',center = False, vmin =0, vmax = 4, filt = True, filter_len=2)
plt.xlabel(None)
plt.ylabel(None)

# cb.ax.set_yticklabels(['a','b','c','d','e'])

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

plt.xlim([1331, 2061])
plt.ylim([1331, 1661])

plt.title('DMSO Contact Map Ave')
plt.savefig('hic234plots/figures/groups/AAave_cropped2_filt2.png')

In [ ]:
plot_matrix(np.log10(AA2ave),cmap='Oranges',center = False, vmin =0, vmax = 4, filt = True, filter_len=2)
plt.xlabel(None)
plt.ylabel(None)

# cb.ax.set_yticklabels(['a','b','c','d','e'])

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

plt.xlim([1331, 2061])
plt.ylim([1331, 1661])

plt.title('2hr Contact Map Ave')
plt.savefig('hic234plots/figures/groups/AA2ave_cropped2_filt2.png')

In [ ]:
plot_matrix(np.log10(AA5ave),cmap='Oranges',center = False, vmin =0, vmax = 4, filt = True, filter_len=2)
plt.xlabel(None)
plt.ylabel(None)

# cb.ax.set_yticklabels(['a','b','c','d','e'])

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

plt.xlim([1331, 2061])
plt.ylim([1331, 1661])

plt.title('5hr Contact Map Ave')
plt.savefig('hic234plots/figures/groups/AA5ave_cropped2_filt2.png')

In [ ]:
plot_matrix(AAave2_0,cmap='bwr',center = True, vmin = -1.25, vcenter = 0, vmax=1.25 , filt = True, filter_len = 0.5)
plt.xlabel(None)
plt.ylabel(None)

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

plt.xlim([1331, 2061])
plt.ylim([1331, 1661])

plt.title('Ave 2hr/DMSO Log2 Ratio')
plt.savefig('hic234plots/figures/groups/AAave2_0_cropped2_filt05.png')

In [ ]:
plot_matrix(AAave5_0,cmap='bwr',center = True, vmin = -1.25,vcenter = 0, vmax=1.25, filt = True, filter_len = 0.5)
plt.xlabel(None)
plt.ylabel(None)

plt.xticks(chromosomes_interval,chrome_plots, rotation = 90)
plt.yticks(chromosomes_interval,chrome_plots)

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)

plt.xlim([1331, 2061])
plt.ylim([1331, 1661])

plt.title('Ave 5hr/DMSO Log2 Ratio')
plt.savefig('hic234plots/figures/groups/AAave5_0_cropped2_filt05.png')

## Individual chromosomes

In [ ]:
AA2ave_chriv = call_region(AA2ave, b'chrIV', b'chrIV')
plot_matrix(np.log10(AA2ave_chriv),cmap='Oranges',center = False, vmin =0, vmax = 4)

plt.xticks(np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6)),np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6))*5)
plt.yticks(np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6)),np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6))*5)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

plt.title('Ave 2hr Contact Map ChrIV')
plt.savefig('hic234plots/figures/chromosomes/AA2ave_chriv.png')

In [ ]:
AAave2_0chriv = call_region(AAave2_0, b'chrIV', b'chrIV')
plot_matrix(AAave2_0chriv,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)

plt.xticks(np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6)),np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6))*5)
plt.yticks(np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6)),np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6))*5)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

plt.title('Ave 2hr/DMSO Log2 Ratio ChrIV')
plt.savefig('hic234plots/figures/chromosomes/AAave2_0_chriv')

In [ ]:
AA5ave_chriv = call_region(AA5ave, b'chrIV', b'chrIV')
plot_matrix(np.log10(AA5ave_chriv),cmap='Oranges',center = False, vmin =0, vmax = 4)

plt.xticks(np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6)),np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6))*5)
plt.yticks(np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6)),np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6))*5)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

plt.title('5hr Contact Map ChrIV')
plt.savefig('hic234plots/figures/chromosomes/AA5ave_chriv.png')

In [ ]:
AAave5_0chriv = call_region(AAave5_0, b'chrIV', b'chrIV')
plot_matrix(AAave5_0chriv,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)

plt.xticks(np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6)),np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6))*5)
plt.yticks(np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6)),np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6))*5)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

plt.title('Ave 5hr/DMSO Log2 Ratio ChrIV')
plt.savefig('hic234plots/figures/chromosomes/AAave5_0_chriv')

In [ ]:
AAave5_0chriv = call_region(AAave5_0, b'chrIV', b'chrIV')
plot_matrix(AAave5_0chriv,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)

plt.xticks(np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6)),np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6))*5)
plt.yticks(np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6)),np.arange(len(AA2ave_chriv),step=int(len(AA2ave_chriv)/6))*5)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

plt.axvline(x=449.75/5, color='black', linestyle='--', linewidth=2)
plt.axhline(y=449.75/5, color='black', linestyle='--', linewidth=2)

plt.title('Ave 5hr/DMSO Log2 Ratio ChrIV')
plt.savefig('hic234plots/figures/chromosomes/AAave5_0_chriv_CENmarked')

In [ ]:
AA2ave_chrii = call_region(AA2ave, b'chrII', b'chrII')
plot_matrix(np.log10(AA2ave_chrii),cmap='Oranges',center = False, vmin =0, vmax = 4)

plt.xticks(np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6)),np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6))*5)
plt.yticks(np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6)),np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6))*5)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

plt.title('Ave 2hr Contact Map ChrII')
plt.savefig('hic234plots/figures/chromosomes/AA2ave_chrii.png')

In [ ]:
AAave2_0chrii = call_region(AAave2_0, b'chrII', b'chrII')
plot_matrix(AAave2_0chrii,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)

plt.xticks(np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6)),np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6))*5)
plt.yticks(np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6)),np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6))*5)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

plt.title('Ave 2hr/DMSO Log2 Ratio ChrII')
plt.savefig('hic234plots/figures/chromosomes/AAave2_0_chrii')

In [ ]:
AA5ave_chrii = call_region(AA5ave, b'chrII', b'chrII')
plot_matrix(np.log10(AA5ave_chrii),cmap='Oranges',center = False, vmin =0, vmax = 4)

plt.xticks(np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6)),np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6))*5)
plt.yticks(np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6)),np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6))*5)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

plt.title('5hr Contact Map ChrII')
plt.savefig('hic234plots/figures/chromosomes/AA5ave_chrii.png')

In [ ]:
AAave5_0chrii = call_region(AAave5_0, b'chrII', b'chrII')
plot_matrix(AAave5_0chrii,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)

plt.xticks(np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6)),np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6))*5)
plt.yticks(np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6)),np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6))*5)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

plt.title('Ave 5hr/DMSO Log2 Ratio ChrII')
plt.savefig('hic234plots/figures/chromosomes/AAave5_0_chrii')

In [ ]:
AAave5_0chrii = call_region(AAave5_0, b'chrII', b'chrII')
plot_matrix(AAave5_0chrii,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)

plt.xticks(np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6)),np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6))*5)
plt.yticks(np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6)),np.arange(len(AA2ave_chrii),step=int(len(AA2ave_chrii)/6))*5)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

plt.axvline(x=283.3/5, color='black', linestyle='--', linewidth=2)
plt.axhline(y=283.3/5, color='black', linestyle='--', linewidth=2)

plt.title('Ave 5hr/DMSO Log2 Ratio ChrII')
plt.savefig('hic234plots/figures/chromosomes/AAave5_0_chrii_CENmarked')

In [ ]:
AAave2_0chrxii = call_region(AAave2_0, b'chrXII', b'chrXII')
plot_matrix(AAave2_0chrxii,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)

plt.xticks(np.arange(len(AAave2_0chrxii),step=int(len(AAave2_0chrxii)/6.1)),np.arange(len(AAave2_0chrxii),step=int(len(AAave2_0chrxii)/6.1))*5)
plt.yticks(np.arange(len(AAave2_0chrxii),step=int(len(AAave2_0chrxii)/6.1)),np.arange(len(AAave2_0chrxii),step=int(len(AAave2_0chrxii)/6.1))*5)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

plt.title('Ave 2hr/DMSO Log2 Ratio ChrXII')
plt.savefig('hic234plots/figures/chromosomes/AAave2_0_chrxii')

In [ ]:
AAave5_0chrxii = call_region(AAave5_0, b'chrXII', b'chrXII')
plot_matrix(AAave5_0chrxii,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)

plt.xticks(np.arange(len(AAave2_0chrxii),step=int(len(AAave2_0chrxii)/6.1)),np.arange(len(AAave2_0chrxii),step=int(len(AAave2_0chrxii)/6.1))*5)
plt.yticks(np.arange(len(AAave2_0chrxii),step=int(len(AAave2_0chrxii)/6.1)),np.arange(len(AAave2_0chrxii),step=int(len(AAave2_0chrxii)/6.1))*5)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

plt.title('Ave 5hr/DMSO Log2 Ratio ChrXII')
plt.savefig('hic234plots/figures/chromosomes/AAave5_0_chrxii')

In [ ]:
AAave5_0chrxii = call_region(AAave5_0, b'chrXII', b'chrXII')
plot_matrix(AAave5_0chrxii,cmap='bwr',center = True, vmin = -1.5,vcenter = 0, vmax=1.5)

plt.xticks(np.arange(len(AAave2_0chrxii),step=int(len(AAave2_0chrxii)/6.1)),np.arange(len(AAave2_0chrxii),step=int(len(AAave2_0chrxii)/6.1))*5)
plt.yticks(np.arange(len(AAave2_0chrxii),step=int(len(AAave2_0chrxii)/6.1)),np.arange(len(AAave2_0chrxii),step=int(len(AAave2_0chrxii)/6.1))*5)
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20)

plt.axvline(x=150.85/5, color='black', linestyle='--', linewidth=2)
plt.axvline(x=460/5, color='black', linestyle='--', linewidth=2)
plt.axhline(y=150.85/5, color='black', linestyle='--', linewidth=2)
plt.axhline(y=460/5, color='black', linestyle='--', linewidth=2)

plt.title('Ave 5hr/DMSO Log2 Ratio ChrXII')
plt.savefig('hic234plots/figures/chromosomes/AAave5_0_chrxii_CEN_rDNA_marked')